In [21]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D

import tensorflow.keras.backend as K
import tensorflow as tf

import cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import os
import tensorflow as tf
import keras as keras
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
import src.proprietary_functions as src

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.datasets import mnist

In [22]:
#dictionary for storing the numpy objects of pairs of images, lables and pairs of images names
arr_dict_X_Y_names = {}

In [23]:
for sample in ['train','valid','test']:
    with open(f'./cropped_numpys/cropped_{sample}_X.npy', 'rb') as f:
         arr_dict_X_Y_names[f'arr_{sample}_X'] = np.load(f)


arr_train_X = arr_dict_X_Y_names['arr_train_X']
arr_valid_X = arr_dict_X_Y_names['arr_valid_X']
arr_test_X = arr_dict_X_Y_names['arr_test_X']

In [24]:
for sample in ['train','valid','test']:
    with open(f'./cropped_numpys/cropped_{sample}_X_names.npy', 'rb') as f:
        arr_dict_X_Y_names[f'{sample}_X_names'] = np.load(f)

train_X_names = arr_dict_X_Y_names['train_X_names']
valid_X_names = arr_dict_X_Y_names['valid_X_names']
test_X_names = arr_dict_X_Y_names['test_X_names']

In [25]:
for sample in ['train','valid','test']:
    with open(f'./cropped_numpys/cropped_{sample}_Y.npy', 'rb') as f:
        arr_dict_X_Y_names[f'arr_{sample}_Y'] = np.load(f)

arr_train_Y = arr_dict_X_Y_names['arr_train_Y']
arr_valid_Y = arr_dict_X_Y_names['arr_valid_Y']
arr_test_Y = arr_dict_X_Y_names['arr_test_Y']

In [26]:
#function creating pairs
def	make_pairs(images, labels, image_names):

	pairImages = []
	pairLabels = []
	pairImagesNames = []
	uniqueClasses = np.unique(labels)

	dict_idx = {i:np.where(labels == i)[0] for i in uniqueClasses}

	for idxA in range(len(images)):
		currentImage = images[idxA]
		label = labels[idxA]
		currentImage_name = image_names[idxA]

		#positive pair
		idxB = np.random.choice(dict_idx[label])
		posImage = images[idxB]
		posImage_name = image_names[idxB]
		pairImages.append([currentImage, posImage])
		pairImagesNames.append([currentImage_name, posImage_name])
		pairLabels.append([1])

		#negative pair
		negLab = np.random.choice([i for i in dict_idx.keys() if i != label])
		negIdx = np.random.choice(dict_idx[negLab])
		negImage = images[negIdx]
		negImage_name = image_names[negIdx]
		pairImages.append([currentImage, negImage])
		pairImagesNames.append([currentImage_name, negImage_name])
		pairLabels.append([0])

	return (np.array(pairImages),np.array(pairLabels)), np.array(pairImagesNames)

In [27]:
(pairTrain, labelTrain), pairNamesTrain = make_pairs(arr_train_X, arr_train_Y, train_X_names)
(pairValid, labelValid), pairNamesValid = make_pairs(arr_valid_X, arr_valid_Y, valid_X_names)
(pairTest, labelTest), pairNamesTest = make_pairs(arr_test_X, arr_test_Y, test_X_names)

In [28]:
import pickle

model_file = "./contrastive_loss_model_.h5"
#model = pickle.load(open("./triplets.sav", "rb"))
model = keras.models.load_model(model_file, compile=False)



In [29]:
predictions = model.predict([pairTest[:, 0], pairTest[:, 1]])

125/125 [==============================] - 56s 430ms/step
